# Import

In [1]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraForPreTraining, ElectraTokenizerFast, ElectraConfig
from torch.optim import AdamW

batch_size = 4

In [2]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

# Wordnet

In [3]:
!pip install nltk
import nltk

# nltk.download('wordnet', os.path.abspath('../working'))

import zipfile
import os

file_path = os.path.abspath("../working/corpora/wordnet.zip")

with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.dirname(file_path))
    
nltk.data.path.append('../working')

from nltk.corpus import wordnet

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


# With RecAdam fine tuning

## Create test Dataset and Dataloader

In [5]:
tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-large-discriminator")

class JsonlDataset(Dataset):
    def __init__(self, filename, is_test=False):
        self.data = [json.loads(line) for line in open(filename, 'r', encoding='utf-8')]
        self.is_test = is_test

    def __len__(self):
        return len(self.data)
    
    def getMerged(self, question, options, article):
        return [
            f"{question.replace('@placeholder', options[i])} [SEP] {article}" for i in range(5)
        ]
    
    def getLabels(self, label):
#         print([(0 if i == label else 1) for i in range(5)])
        return torch.tensor([(0 if i == label else 1) for i in range(5)])
    
    def __getitem__(self, idx):
        item = self.data[idx]
        article = item['article']
        question = item['question']
        options = [item[f'option_{i}'] for i in range(5)]
        
        merged = self.getMerged(question, options, article)
        
        _input = tokenizer(
            merged,
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )
        
        input_ids = _input["input_ids"]
        attention_mask = _input["attention_mask"]
        
        origin = f"{question.replace('@placeholder', '[MASK]')} [SEP] {article}"
        
        origin_ids = tokenizer(
            origin,
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )["input_ids"]
        
        option_position = (origin_ids == tokenizer.mask_token_id).nonzero().tolist()[0][1]
        
        if self.is_test:
            return {
                'article': article,
                'question': question,
                'options': options,
                'input_ids': input_ids,
                "attention_mask": attention_mask,
                'merged': merged,
                'option_position': option_position
            }
        else:
            return {
                'article': article,
                'question': question,
                'options': options,
                'label': item['label'],
                'input_ids': input_ids,
                "attention_mask": attention_mask,
                'labels': self.getLabels(item['label']),
                'merged': merged,
                'option_position': option_position
            }

In [6]:
task1_test_dataset = JsonlDataset('../input/semevaldataset/trail_data/Task_1_Imperceptibility.jsonl')
task1_test_loader = DataLoader(task1_test_dataset, batch_size=batch_size, shuffle=True)


task2_test_dataset = JsonlDataset('../input/semevaldataset/trail_data/Task_2_Nonspecificity.jsonl')
task2_test_loader = DataLoader(task2_test_dataset, batch_size=batch_size, shuffle=True)

In [7]:
def eval(model, test_loader):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model = model.to(device)
    model.eval()
    
    total = 0
    correct = 0
    for i, batch in enumerate(test_loader):
        with torch.no_grad():
            _input_ids = batch['input_ids'].to(device)
            _labels = batch['labels']
            _label = batch['label']
            _attention_mask = batch['attention_mask'].to(device)
            _option_position = batch['option_position']
            
            for i in range(_input_ids.shape[0]):
                input_ids = _input_ids[i]
                labels = _labels[i]
                label = _label[i]
                attention_mask = _attention_mask[i]
                option_position = _option_position[i]
                
                # input_ids shape: torch.Size([5, 512])
                # labels shape: torch.Size([5]
                # print('input ids: ', input_ids.shape, 'labels: ', labels.shape)
            
                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                
                # logit shape: torch.Size([5, 512])
                # print('logits: ', logits.shape)
                
                # probabilities = torch.sigmoid(logits)
                probabilities = logits[:, option_position]
                # print(probabilities, label)
                
                most_likely_idx = probabilities.argmin().item()
                
                # print(most_likely_idx, label)
                correct += (most_likely_idx == label.item())
                total += 1

    print(correct / total)

In [8]:
# config = ElectraConfig("google/electra-large-discriminator")
discriminator = ElectraForPreTraining.from_pretrained("../input/semevalmodelelectra/task1_electra_recadam_last.bin")
print('Task1: ')
eval(discriminator, task1_test_loader)
print('Task1 cross: ')
eval(discriminator, task2_test_loader)

discriminator = ElectraForPreTraining.from_pretrained("../input/semevalmodelelectra/task2_electra_recadam_last.bin")
print('Task2: ')
eval(discriminator, task2_test_loader)
print('Task2 cross: ')
eval(discriminator, task1_test_loader)

Task1: 
0.896
Task1 cross: 
0.926
Task2: 
0.928
Task2 cross: 
0.896


## Without any Fine tuning: Task1, Task2

In [9]:
# Evaluation Task1, Cross1, Task2, Cross2
discriminator = ElectraForPreTraining.from_pretrained("google/electra-large-discriminator")

print('Task1: ')
eval(discriminator, task1_test_loader)
print('Task2: ')
eval(discriminator, task2_test_loader)

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForPreTraining: ['electra.embeddings_project.bias', 'electra.embeddings_project.weight']
- This IS expected if you are initializing ElectraForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Task1: 
0.898
Task2: 
0.923


# With Wordnet

In [10]:
class JsonlDataset2(JsonlDataset):
    def getMerged(self, question, options, article):
        wordnet_meaning = []
        for i in range(5):
            synsets = wordnet.synsets(options[i])
            if len(synsets):
                wordnet_meaning.append(f"{synsets[0].definition()}")
            else:
                wordnet_meaning.append("")
        return [
            f"{question.replace('@placeholder', options[i])} [SEP] {wordnet_meaning[i]} [SEP] {article}" for i in range(5)
        ]

In [11]:
task1_test_dataset = JsonlDataset2('../input/semevaldataset/trail_data/Task_1_Imperceptibility.jsonl')
task1_test_loader = DataLoader(task1_test_dataset, batch_size=batch_size, shuffle=True)

task2_test_dataset = JsonlDataset2('../input/semevaldataset/trail_data/Task_2_Nonspecificity.jsonl')
task2_test_loader = DataLoader(task2_test_dataset, batch_size=batch_size, shuffle=True)

## Task1, Task2

In [12]:
print('Task1: ')
eval(discriminator, task1_test_loader)
print('Task2: ')
eval(discriminator, task2_test_loader)

Task1: 
0.85
Task2: 
0.895
